In [ ]:
# from datasets import Dataset, DatasetDict, ClassLabel
# import numpy as np

# # Define your data dictionary
# data_dict = {
#     "image": [np.random.random((128, 128, 3)), np.random.random((128, 128, 3)), np.random.random((128, 128, 3))],
#     "label": [0, 1, 2]  # Assuming label indices correspond to class labels
# }

# # Define the ClassLabel mapping
# class_label = ClassLabel(num_classes=3, names=["Cat", "Dog", "Bird"])

# Create a Dataset using DatasetDict
# dataset = DatasetDict({
#     "train": Dataset.from_dict(data_dict, features={"image": "image", "label": class_label})
# })

# # Access and print the dataset
# print(dataset)


In [ ]:
import tifffile
import os
from PIL import Image
import numpy as np

# Define the ClassLabel mapping
from datasets import ClassLabel

class_label = ClassLabel(num_classes=2, names=["nonprogressor", "progressor"])

# Function to create PIL image
def create_pil_image(new_image, new_image_path):
    # Convert the data type to uint8
    new_image = new_image.astype(np.uint8)
    print("new_image shape:", new_image.shape)
    print("new_image dtype:", new_image.dtype)

    print("Min value:", np.min(new_image))
    print("Max value:", np.max(new_image))

    # print("new_image content:", new_image[:3, :3, :3])  # Print the first 3x3x3 subarray

    new_image_pil = Image.fromarray(new_image, mode="RGB")
    print("new_image_pil dtype: {} ".format(type(new_image_pil)))
    # new_image_pil = Image.fromarray(new_image)
    new_image_pil.save(new_image_path)
    return(new_image_pil)

# call this for each one of the labels
def load_47_channel_tif_to_dataset_object(list_of_images_files, label):
    # Define dimensions
    original_channels = 47
    original_rows = 1024
    new_channels = 3
    new_rows = 4053
    new_cols = 4053
    data_list = list()

    # Initialize new_image array outside the loop
    new_image = np.zeros((new_channels, new_rows, new_cols), dtype=np.uint8)
    
    for image_path in list_of_images_files:
        # Read the image
        original_image = tifffile.imread(image_path)
        
        # Iterate through original image channels and rows
        for channel in range(original_channels):
            for row in range(original_rows):
                new_row = (channel * original_rows + row) % new_rows
                if new_row < new_rows:
                    new_image[channel % new_channels, new_row, :original_image.shape[2]] = original_image[channel, row, :]

        # Create PIL image and save it
        new_image_filename = os.path.basename(image_path).replace('.tif', '.jpg')
        new_image_save_path = os.path.join(os.path.dirname(image_path), new_image_filename)
        new_image_pil = create_pil_image(new_image, new_image_save_path)

        # Append paths and labels to data_dict
        data_item = {
            'image': new_image_pil,
            'image_file_path': new_image_save_path,
            'labels': class_label.str2int(label)
        }
        data_list.append(data_item)
        
    return data_list

# # Example usage
# list_of_images_files = ["path/to/image1.tif", "path/to/image2.tif"]
# label = "progressor"
# data_dict = load_47_channel_tif_to_dataset_object(list_of_images_files, label)
# print(data_dict)


In [ ]:
def preprocess_image(example):
    image = example['image']  # Assuming 'image' is the key in your dataset containing image data
    inputs = feature_extractor(images=image, return_tensors="pt")
    return inputs

# preprocessed_dataset = dataset.map(preprocess_image)
